In [1]:
!pip install python-whois
import pandas as pd
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime
import requests

In [14]:
!wget http://data.phishtank.com/data/online-valid.csv

URL transformed to HTTPS due to an HSTS policy
--2022-12-15 20:47:31--  https://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 2606:4700::6810:654b, 2606:4700::6811:b155, 104.16.101.75, ...
Connecting to data.phishtank.com (data.phishtank.com)|2606:4700::6810:654b|:443... connected.
HTTP request sent, awaiting response... 429 Too Many Requests
2022-12-15 20:47:32 ERROR 429: Too Many Requests.



In [2]:
data = pd.read_csv("online-valid.csv")
data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7978631,https://inomueblessa.01336587.repl.co/?gclid=E...,http://www.phishtank.com/phish_detail.php?phis...,2022-12-16T01:07:01+00:00,yes,2022-12-16T01:12:20+00:00,yes,Other
1,7978627,https://datingcrimepreventionagreement.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-16T00:43:02+00:00,yes,2022-12-16T00:51:59+00:00,yes,Other
2,7978625,https://boagh-symbieuntz-smip.yolasite.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-16T00:36:36+00:00,yes,2022-12-16T00:42:10+00:00,yes,Other
3,7978624,http://ttec.ink,http://www.phishtank.com/phish_detail.php?phis...,2022-12-16T00:33:43+00:00,yes,2022-12-16T00:42:10+00:00,yes,Other
4,7978607,https://projetosarea51.000webhostapp.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-15T23:53:05+00:00,yes,2022-12-16T00:07:01+00:00,yes,Other


In [3]:
phishurl = data.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7973904,https://www.oricoe-co-jp.oriseco.bqodxg.top/ai...,http://www.phishtank.com/phish_detail.php?phis...,2022-12-12T05:27:13+00:00,yes,2022-12-12T05:32:51+00:00,yes,Other
1,7977133,https://de-dkb.cloud/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-14T16:48:16+00:00,yes,2022-12-14T16:52:05+00:00,yes,Das kann Bank
2,7967376,http://www.amazcon-co-jp.amazocm.sdafgj.top/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-06T04:11:05+00:00,yes,2022-12-06T05:23:01+00:00,yes,Amazon.com
3,7976914,https://sipp.umtas.ac.id/uploads/fonts/languag...,http://www.phishtank.com/phish_detail.php?phis...,2022-12-14T15:27:49+00:00,yes,2022-12-14T15:42:12+00:00,yes,Other
4,7521967,https://disrupto.xyz,http://www.phishtank.com/phish_detail.php?phis...,2022-05-20T09:43:03+00:00,yes,2022-08-15T08:44:06+00:00,yes,Other


In [4]:
data = pd.read_csv("Benign_list_big_final.csv")
data.columns = ['URLs']
data.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [5]:
legiurl = data.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [6]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [7]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re
def getDomain(url):  
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain
def havingIP(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip
def haveAtSign(url):
    if "@" in url:
        at = 1    
    else:
        at = 0    
    return at
def getLength(url):
    if len(url) < 54:
        length = 0            
    else:
        length = 1            
    return length
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 1
    else:
        return 0
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1           
    else:
        return 0
def web_traffic(url):
    try:
        url = urllib.parse.quote(url)
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
            "REACH")['RANK']
        rank = int(rank)
    except TypeError:
        return 1
    if rank > 100000:
        return 1
    else:
        return 0
def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
        try:
            creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            age = 1
        else:
            age = 0
    return age
def domainEnd(domain_name):
    expiration_date = domain_name.expiration_date
    if isinstance(expiration_date,str):
        try:
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if (expiration_date is None):
        return 1
    elif (type(expiration_date) is list):
        return 1
    else:
        today = datetime.now()
        end = abs((expiration_date - today).days)
        if ((end/30) < 6):
            end = 0
        else:
            end = 1
    return end
def iframe(response):
    if response == "":
        return 1
    else:
        if re.findall(r"[<iframe>|<frameBorder>]", response.text):
            return 0
        else:
            return 1
def mouseOver(response): 
    if response == "" :
        return 1
    else:
        if re.findall("<script>.+onmouseover.+</script>", response.text):
            return 1
        else:
            return 0
def rightClick(response):
    if response == "":
        return 1
    else:
        if re.findall(r"event.button ?== ?2", response.text):
            return 0
        else:
            return 1
def forwarding(response):
    if response == "":
        return 1
    else:
        if len(response.history) <= 2:
            return 0
        else:
            return 1
def featureExtraction(url,label):
    features = []
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(prefixSuffix(url))
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1

    features.append(dns)
    features.append(web_traffic(url))
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))
    try:
        response = requests.get(url)
    except:
        response = ""
    features.append(iframe(response))
    features.append(mouseOver(response))
    features.append(rightClick(response))
    features.append(forwarding(response))
    features.append(label)
  
    return features


In [ ]:
legi_features = []
label = 0

for i in range(0, 5000):
    url = legiurl['URLs'][i]
    legi_features.append(featureExtraction(url,label))
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)

Error trying to connect to socket: closing socket - [Errno 8] nodename nor servname provided, or not known
Error trying to connect to socket: closing socket - [Errno 8] nodename nor servname provided, or not known


In [ ]:
phish_features = []
label = 1
for i in range(0, 5000):
    url = phishurl['url'][i]
    phish_features.append(featureExtraction(url,label))
phishing = pd.DataFrame(phish_features, columns= feature_names)

In [ ]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.to_csv('urldata.csv', index=False)